# Evaluating Numerical Expressions
---
In this guided project, we will use stacks to implement an algorithm that can evaluate numerical expressions. Calculating the result of a complex numerical expression isn't something that a computer processor can do right out of the box. Behind the scenes, Python uses an algorithm to evaluate this expression.

The goal of this guided project is to use the stack data structure that we've worked with in this course to implement an algorithm that can evaluate complex numerical expressions.

### 1. Introduction
We will start this project by importing back a class we've made previously, which is called `LinkedList` available from `linked_list.py`

In [1]:
# Importing the LinkedList class
from linked_list import LinkedList

Our project will also need an implementation of `LinkedList` which is called `Stack`, we will bring it in our workspace.

In [2]:
# Creating the stack implementation
class Stack(LinkedList):
    
    def push(self, data):
        self.append(data)

    def peek(self):
        return self.tail.data

    def pop(self):
        ret = self.tail.data
        if self.length == 1:
            self.tail = self.head = None
        else:
            self.tail = self.tail.prev
            self.tail.next = None
        self.length -= 1
        return ret

### 2. Implementing the tokenize function
Computers have an easier time of evaluating a **postfix notation** (instead of writing `1 + 2`, it is easier to evaluate `1 2 +`) instead of **infix notation**. First we will create a function which will split a string into individual elements which will be called as `tokenize()`.

In [3]:
# Defining tokenize
def tokenize(string):
    return string.split()

# Testing the function
print(tokenize("12 2 4 + / 21 *"))

['12', '2', '4', '+', '/', '21', '*']


### 3. Processing an Operator
Now we will try and implement simple operators using `stack` class. A thing to note is the other of the operands, as we know `1 - 2` is not equal to `2 - 1`, thus keeping the order of the operantds is crucial.

In [4]:
# Defining substract
def process_minus(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top - top
    stack.push(result)

# Defining add
def process_plus(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top + top
    stack.push(result)
    
# Defining multiplication
def process_times(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top * top
    stack.push(result)
    
# Defining division
def process_divide(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top / top
    stack.push(result)
    
# Defining power
def process_pow(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top ** top
    stack.push(result)

### 4. Evaluating postfix expressions
Here are the steps we need to follow to implement the evaluate_postfix() function.

1. Initialize an empty stack.
2. Tokenize the expression using the tokenize() function.
3. For each token, do:
    - If the token an operator, call the corresponding function to process it. For example, if we find a + we call the process_plus() function.
    - Otherwise (the token is a number) and we push that number to the top of the stack. Since each token is a string, we'll need to convert it to a float first.
4. Return the value that is left in the stack.

In [5]:
# Defining the evaluate postfix function
def evaluate_postfix(expression):
    tokens = tokenize(expression)
    stack = Stack()
    for token in tokens:
        if token == "+":
            process_plus(stack)
        elif token == "-":
            process_minus(stack)
        elif token == "*":
            process_times(stack)
        elif token == "/":
            process_divide(stack)
        elif token == "**":
            process_pow(stack)
        else:
            stack.push(float(token))
    return stack.pop()

# Testing the function
expressions = [
    "4 6 -",
    "4 1 2 9 3 / * + 5 - *",
    "1 2 + 3 -",
    "1 2 - 3 +",
    "10 3 5 * 16 4 - / +",
    "5 3 4 2 - ** *",
    "12 2 4 + / 21 *",
    "1 1 + 2 **",
    "1 1 2 ** +"
]

for expression in expressions:
    print(evaluate_postfix(expression))

-2.0
8.0
0.0
2.0
11.25
45.0
42.0
4.0
2.0


### 5. Operator Precedence in Infix Notation
We will now begin to implement brackets into our equation. In mathematics, operators enclosed in brackets have a higher order than those which are not (or simply operator precedence).

We will try and implement the [**Shunting-yard Algorithm**](https://en.wikipedia.org/wiki/Shunting-yard_algorithm#:~:text=In%20computer%20science%2C%20the%20shunting,abstract%20syntax%20tree%20(AST).), thus we need to compare the precedence of operators.

In [6]:
# Defining operator precedence
precedence = {
    "+": 1,
    "-": 1,
    "*": 2,
    "/": 2,
    "**": 3
}

# Testing the dictionary
print(precedence["/"] < precedence["-"])
print(precedence["+"] < precedence["*"])
print(precedence["+"] < precedence["-"])
print(precedence["/"] < precedence["**"])

False
True
False
True


### 6. Processing tokens in infix to postfix conversions
By the end, we will be creating a function called `infix_to_postfix()`. This function will implement the Shunting-yard algorithm. This algorithm is similar to the `evaluate_postfix()` function we've implemented before. It starts by tokenizing the postfix expression, and then it processes the tokens one by one using a stack. It builds the postfix expression by keeping track of a list named `postfix`, which will contain the list of tokens in postfix order.
#### 6.1. Opening Parenthesis
We will create a function to push the string "(" into the stack.

In [7]:
# Defining a function for the opening parenthesis
def process_opening_parenthesis(stack):
    stack.push("(")

#### 6.2. Closing Parenthesis
We will create a function which:
1. While the top of the stack isn't an opening parenthesis, (, pop the top element, and append it to the postfix token list.
2. Pop the opening parentheses out of the stack at the end.

In [8]:
# Defining a function for the closing parenthesis
def process_closing_parenthesis(stack, postfix):
    while stack.peek() != "(":
        postfix.append(stack.pop())
    stack.pop()

#### 6.3. Operators
This is the thought process for operators:
1. While the top of the stack is also an operator with a precedence greater than or equal to this operator, pop the top element and append it to the postfix token list.
2. Push the current operator to the top of the stack.

In [9]:
# Defining the operator function with precedence
def process_operator(stack, postfix, operator):
    while len(stack) > 0 and stack.peek() in precedence and precedence[stack.peek()] >= precedence[operator]:
        postfix.append(stack.pop())
    stack.push(operator)

#### 6.4. Numbers
We will create a function called `process_number()` which will push the token into the postfix token list.

In [10]:
# Defining the number function
def process_number(postfix, number):
    postfix.append(number)

### 7. The Shunting-yard Algorithm

We now have all the pieces we need to implement the `infix_to_postfix()` function that converts an expression from infix notation to postfix notation.

This function will work as follows:

1. We start by splitting the expression into tokens using the `tokenize()` function.
2. We initialize an empty stack.
3. We initialize an empty postfix token list.
4. Iterate over all tokens, and for each, do the following:
    - If the token is "(", we call the `process_opening_parenthesis()` function.
    - If the token is ")", we call the `process_closing_parenthesis()` function.
    - If the token is an operator, we call the `process_operator()` function.
    - Otherwise, the token is a number, and we call the `process_number()` function.
5. After processing all tokens, we use a while loop to pop the remaining stack element into the postfix token list.
6. Use the `str.join()` method to convert the postfix token list into a string.

In [11]:
# Defining the function
def infix_to_postfix(expression):
    tokens = tokenize(expression)
    stack = Stack()
    postfix = []
    for token in tokens:
        if token == "(":
            process_opening_parenthesis(stack)
        elif token == ")":
            process_closing_parenthesis(stack, postfix)
        elif token in precedence:
            process_operator(stack, postfix, token)
        else:
            process_number(postfix, token)
    while len(stack) > 0:
        postfix.append(stack.pop())
    return " ".join(postfix)

### 8. Evaluating Infix expressions
We now have a function that can transform an infix expression into postfix notation and a function that can evaluate an expression in postfix notation. By combining the two, we can write a function named `evaluate()` that returns the value of an expression in infix notation.

In [12]:
# Defining the evaluate function
def evaluate(expression):
    postfix_expression = infix_to_postfix(expression)
    return evaluate_postfix(postfix_expression)

# Testing the function
expressions = [
    "1 + 1",
    "1 * ( 2 - ( 1 + 1 ) )",
    "4 * ( 1 + 2 * ( 9 / 3 ) - 5 )",
    "10 + 3 * 5 / ( 16 - 4 * 1 )",
    "2 * 2 * 2 * 2 * 2 * 2 * 2 * 2",
    "2 ** 2 ** 2 ** 2 ** 2",
    "( 1 - 2 ) / ( 3 - 5 )",
    "9 / 8 * 8",
    "64 / ( 8 * 8 )",
]

for expression in expressions:
    print(evaluate(expression))

2.0
0.0
8.0
11.25
256.0
65536.0
0.5
9.0
1.0
